In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed

In [56]:
A = fill(1, (3,4,5))

3×4×5 Array{Int64,3}:
[:, :, 1] =
 1  1  1  1
 1  1  1  1
 1  1  1  1

[:, :, 2] =
 1  1  1  1
 1  1  1  1
 1  1  1  1

[:, :, 3] =
 1  1  1  1
 1  1  1  1
 1  1  1  1

[:, :, 4] =
 1  1  1  1
 1  1  1  1
 1  1  1  1

[:, :, 5] =
 1  1  1  1
 1  1  1  1
 1  1  1  1

In [ ]:
function rinv(R, )

In [46]:
inv(Fupdated.R)

7×7 UpperTriangular{Float64,Array{Float64,2}}:
 -0.1   0.395296   0.425872     1.85994  -0.142374   -0.552076     1.77092
   ⋅   -0.407748  -0.00535497  -1.90514   0.126601    0.139192    -0.408838
   ⋅     ⋅        -0.432405    -1.85796   0.113328    0.118634    -0.414622
   ⋅     ⋅          ⋅           1.90481  -0.0987392  -0.117106    -0.939366
   ⋅     ⋅          ⋅            ⋅        0.100469   -0.00617912  -0.0106442
   ⋅     ⋅          ⋅            ⋅         ⋅          0.424289    -1.38723
   ⋅     ⋅          ⋅            ⋅         ⋅           ⋅           1.37881

In [58]:
@testset "Test updateQRscaling" begin
    Nx = 3
    Ne = 8
    m = 20

    idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


    Nψ = 6
    coeff = [ 0.20649582065364197;
             -0.5150990160472986;
              2.630096893080717;
              1.13653076177397;
              0.6725837371023421;
             -1.3126095306624133]
    C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

    Ne = 100


    # The QR decomposition is not unique!

    X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

    L = LinearTransform(X)
    transform!(L, X)
    S = Storage(C.I.f, X)
    F = QRscaling(S)
    newidx = [1 1 1]

    Snew = update_storage(S, X, newidx)
    Fupdated = updateQRscaling(F, Snew)

    Fnew = QRscaling(Snew)

    @test norm(Fupdated.D - Fnew.D)<1e-8
    @test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

    @test norm(Fupdated.R.data'*Fupdated.R.data - Fnew.R.data'*Fnew.R.data)<1e-8

    @test norm(inv(Fupdated.R) - Fupdated.Rinv.data)<1e-8
    @test norm(inv(Fupdated.U) - Fupdated.Uinv.data)<1e-8

    @test norm(Fupdated.L2Uinv - Fupdated.Uinv'*Fupdated.Uinv)<1e-8

end

Test Summary:        | Pass  Total
Test updateQRscaling |    6      6


Test.DefaultTestSet("Test updateQRscaling", Any[], 6, false)

In [53]:
norm(Fupdated.Rinv*Fupdated.R -I)

5.633900080430137e-16

In [54]:
norm(Fupdated.R*Fupdated.Rinv -I)

1.350890776860776e-15

In [49]:
Fnew.Rinv.data

7×7 Array{Float64,2}:
 -0.1   0.395296   0.425872     1.85994  -0.142374   -0.552076    -1.77092
 -0.0  -0.407748  -0.00535497  -1.90514   0.126601    0.139192     0.408838
 -0.0  -0.0       -0.432405    -1.85796   0.113328    0.118634     0.414622
  0.0   0.0        0.0          1.90481  -0.0987392  -0.117106     0.939366
  0.0   0.0        0.0          0.0       0.100469   -0.00617912   0.0106442
  0.0   0.0        0.0          0.0       0.0         0.424289     1.38723
 -0.0  -0.0       -0.0         -0.0      -0.0        -0.0         -1.37881

In [ ]:
Fnew

In [27]:
Fupdated.Rinv*Fupdated.R -I

7×7 UpperTriangular{Float64,Array{Float64,2}}:
 0.0  0.0  0.0  -1.11022e-16  -2.22045e-16   9.71445e-17  -3.46945e-16
  ⋅   0.0  0.0   0.0          -4.44089e-16   1.11022e-16  -2.22045e-16
  ⋅    ⋅   0.0   1.11022e-16   2.22045e-16  -1.11022e-16   3.33067e-16
  ⋅    ⋅    ⋅    0.0           0.0           0.0           2.22045e-16
  ⋅    ⋅    ⋅     ⋅            0.0           0.0           3.03577e-18
  ⋅    ⋅    ⋅     ⋅             ⋅            0.0          -1.11022e-16
  ⋅    ⋅    ⋅     ⋅             ⋅             ⋅           -2.0

In [25]:
inv(Fupdated.R)

7×7 UpperTriangular{Float64,Array{Float64,2}}:
 -0.1  -0.397168   0.418147   -1.80275  -0.0474094  -0.0322345   0.158918
   ⋅    0.409564  -0.0227366   1.86306   0.0536014  -0.376065    1.11229
   ⋅     ⋅        -0.408565    1.76442   0.0925128  -0.321914    1.04124
   ⋅     ⋅          ⋅         -1.82256  -0.108877    0.332872   -2.30807
   ⋅     ⋅          ⋅           ⋅        0.101518    0.0145133  -0.00424558
   ⋅     ⋅          ⋅           ⋅         ⋅          0.407931   -1.32859
   ⋅     ⋅          ⋅           ⋅         ⋅           ⋅          1.3294

In [24]:
Fupdated.Rinv.data

7×7 Array{Float64,2}:
 -0.1  -0.397168   0.418147   -1.80275  -0.0474094  -0.0322345   0.158918
  0.0   0.409564  -0.0227366   1.86306   0.0536014  -0.376065    1.11229
 -0.0  -0.0       -0.408565    1.76442   0.0925128  -0.321914    1.04124
 -0.0  -0.0       -0.0        -1.82256  -0.108877    0.332872   -2.30807
  0.0   0.0        0.0         0.0       0.101518    0.0145133  -0.00424558
  0.0   0.0        0.0         0.0       0.0         0.407931   -1.32859
  0.0   0.0        0.0         0.0       0.0         0.0        -1.3294

In [14]:
@testset "Test updateQRscaling" begin
    Nx = 3
    Ne = 8
    m = 20

    idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


    Nψ = 6
    coeff = [ 0.20649582065364197;
             -0.5150990160472986;
              2.630096893080717;
              1.13653076177397;
              0.6725837371023421;
             -1.3126095306624133]
    C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

    Ne = 100


    # The QR decomposition is not unique!

    X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

    L = LinearTransform(X)
    transform!(L, X)
    S = Storage(C.I.f, X)
    F = QRscaling(S)
    newidx = [1 1 1]

    Snew = update_storage(S, X, newidx)
    Fupdated = updateQRscaling(F, Snew)

    Fnew = QRscaling(Snew)

    @test norm(Fupdated.D - Fnew.D)<1e-8
    @test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

    @test norm(Fupdated.R.data'*Fupdated.R.data - Fnew.R.data'*Fnew.R.data)<1e-8
    @test norm(Fupdated.Rinv.data'*Fupdated.Rinv.data - Fnew.Rinv.data'*Fnew.Rinv.data)<1e-8

    @test norm(Fupdated.U.data'*Fupdated.U.data - Fnew.U.data'*Fnew.U.data)<1e-8  
    @test norm(Fupdated.Uinv.data'*Fupdated.Uinv.data - Fnew.Uinv.data'*Fnew.Uinv.data)<1e-8  

    @test norm(Fupdated.L2Uinv - Fnew.L2Uinv)<1e-8

end

Test Summary:        | Pass  Total
Test updateQRscaling |    7      7


Test.DefaultTestSet("Test updateQRscaling", Any[], 7, false)

### Integration

In [24]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

LoadError: AssertionError: Size of the array of multi-indices idx is wrong

In [22]:
M = HermiteMap(m, X);

In [9]:
@time optimize(M, X, 10; P = serial);

 11.061632 seconds (49.90 M allocations: 19.060 GiB, 7.20% gc time)


In [10]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = serial, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)
